In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

data		    gpi.py		 Optimize_S.py	__pycache__
generate_graphs.py  Optimize_alpha_k.py  Optimize_W.py


In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries
!pip install nimfa -t /home/aistudio/work/pkgs/

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 102kB 275kB/s ta 0:00:011
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 4.7MB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 15.7MB 417kB/s eta 0:00:01
     |████████████████████████████████| 38.1MB 275kB/s eta 0:00:01
ERROR: blackhole 1.0.1 has requirement numpy<=1.19.5, but you'll have numpy 1.21.4 which is incompatible.


In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [ ]:
import numpy as np
import math
from scipy.linalg import orth
from numpy.random import rand
from work.Optimize_W import Optimize_W
from work.Optimize_S import Optimize_S
from work.Optimize_alpha_k import Optimize_alpha_k
from work.generate_graphs import generate_graphs
from work.preprocessing import data_preprocess

In [ ]:
'''
数据集url
'''
from work.preprocessing import preprocess_normal_std
data_path = 'work/data/Indian_pines_corrected.mat'
labels_path = 'work/data/Indian_pines_gt.mat'
data_mat, labels_mat = preprocess_normal_std(data_path, labels_path)

In [ ]:

def SPCA_AMGLUSBS(data_path, labels_path, alpha, beta, gamma, d):
    '''
    X: 数据矩阵,论文中shape为[b,mn*mn],代码中实际为[mn*mn,b]
    d: 选择的波段子集中包含的波段个数
    '''
    W = 20
    H = 20
    X, y = data_preprocess(data_path, labels_path, W, H)
    
    mn = X.shape[0]
    n_neighbors = 20
    b = X.shape[1]

    #正则化子图Si
    Si = generate_graphs(X, n_neighbors)
    K = Si.shape[0]
   
    #初始化S
    S = np.sum(Si, axis=0) / K

    #初始化系数参数   
    gamma = 1 / K
    alpha_k = np.ones(K)
    alpha_k = alpha_k / K
    iteration = 0
    iter_max = 5
    W_init = orth(np.random.randn(b,d))
    #diff = 1
    #obj = np.zeros(iter_max)
    '''
    #SS = Sij * log(Sij)
    SS = np.zeros(K)
    for k in range(K):
        index = np.where(Si[k] > 0)
        SS[k] = np.sum(Si[k][index] * np.log(Si[k][index]))                     #math.log()要求输入必须是标量,np.log可以对向量进行log计算
    '''
    while iteration < iter_max:
      
        # update W
        W = Optimize_W(X, W_init, S, d, alpha, beta, 1)

        # update S
        S = Optimize_S(X, W, Si, beta, gamma, alpha_k)
       
        # update alpha_k
        alpha_k = Optimize_alpha_k(Si, S)

        '''
        index  = np.where(S > 0)
        '''
        #B = np.zeros([mn,mn])
        #for i in range(mn):
        #    for j in range(mn):
        #        if S[i,j] > 0:
        #            B[i,j] = np.sum(alpha_k[k] ** 2 * Si[k,i,j] for k in range(K))
        '''
        B = np.sum(alpha_k[k] ** 2 * Si[k][index] for k in range(K))
        tmp1 = np.sum(alpha_k[k] ** 2 * SS[k] for k in range(K))
        tmp2 = np.sum(B * np.log(S[index]))
        
        D = np.diag(np.sum(S, axis=1))
        L = D - S
        
        # objective function value
        obj[iteration] = -np.trace(W.T @ X.T @ X @ W) + alpha * np.sum(np.linalg.norm(w) for w in W) + beta * np.trace(W.T @ X.T @ L @ X @ W) + gamma * (tmp1 - tmp2)
        if iteration >= 1:
            diff = obj[iteration-1] - obj[iteration]
            print(diff)
        '''
        iteration = iteration + 1
    score = np.sum(W * W, axis=1)
    index_sorted = np.argsort(score)        #升序排列,需要逆序处理
    index_sorted = index_sorted[-d:][::-1]  #选择后d个

    return index_sorted

array([14, 12])

In [ ]:
'''
result_svm = np.zeros([6,6,6])
result_knn = np.zeros([6,6,6])
for i, alpha in enumerate([0.001,0.01,0.1,1,10,100]):
    for j, beta in enumerate([0.001,0.01,0.1,1,10,100]):
        for k, gamma in enumerate([0.001,0.01,0.1,1,10,100]):
'''
def testParameter(alpha, beta, gamma,data_mat, labels_mat):
    acc_s = 0
    acc_k = 0
    for i in range(10):
        #print('alpha:',alpha,'beta:',beta,'gamma:',gamma)
        sorted_index = SPCA_AMGLUSBS(X, 10, 1, 100, 15)
        #print(sorted_index)
        # gamma不能大
        data_mat_new, labels_mat_new = data_mat[:,sorted_index], labels_mat
        num = 10
        acc_svm = 0
        acc_knn = 0
        for o in range(num):
            acc_svm += eval_band_SVM(data_mat_new, labels_mat_new)
            acc_knn += eval_band_KNN(data_mat_new, labels_mat_new)

        acc_svm /= num
        acc_knn /= num
        acc_s += acc_svm
        acc_k += acc_knn
        #result_svm[i,j,k] = acc_svm
        #result_knn[i,j,k] = acc_knn
        #print('acc by svm:', acc_svm)
        #print('acc by knn:', acc_knn)
    print(acc_s/10)    
    print(acc_k/10)

for G in [0.01,0.05,0.1, 1 ,10, 100, 200, 500, 1000]:
    testParameter(100,1,G,data_mat,labels_mat)

In [ ]:
# 稀疏PCA及自适应多图学习的评估
from work.estimate import eval_band_SVM
from work.estimate import eval_band_KNN
acc_s = 0
acc_k = 0
kappa_s = 0
kappa_k = 0
for i in range(10):
    #print('alpha:',alpha,'beta:',beta,'gamma:',gamma)
    # 提高alpha降低性能，提高beta有助于提高SVM，提高gamma有助于提高KNN
    sorted_index = SPCA_AMGLUSBS(data_path, labels_path,1,300,70,35)
 
    # gamma不能大
    data_mat_new, labels_mat = data_mat[:,sorted_index], labels_mat
    num = 20
    acc_svm = 0
    acc_knn = 0
    kappa_svm = 0
    kappa_knn = 0
    for o in range(num):
        a, b = eval_band_SVM(data_mat_new, labels_mat)
        c, d = eval_band_KNN(data_mat_new, labels_mat)
        acc_svm += a
        kappa_svm += b
        acc_knn += c
        kappa_knn += d
    acc_svm /= num
    acc_knn /= num
    kappa_svm /= num
    kappa_knn /= num
    acc_s += acc_svm
    acc_k += acc_knn
    kappa_s += kappa_svm
    kappa_k += kappa_knn
    #result_svm[i,j,k] = acc_svm
    #result_knn[i,j,k] = acc_knn
    print('acc by svm:', acc_svm)
    print('acc by knn:', acc_knn)
    print('kappa by svm:', kappa_svm)
    print('kappa by knn:', kappa_knn)
print(acc_s/10)    
print(acc_k/10)
print(kappa_s/10)
print(kappa_k/10)


In [ ]:
def fun(l):
    acc_s = 0
    acc_k = 0
    kappa_s = 0
    kappa_k = 0
    for i in range(10):
        #print('alpha:',alpha,'beta:',beta,'gamma:',gamma)
        # 提高alpha降低性能，提高beta有助于提高SVM，提高gamma有助于提高KNN
        sorted_index = SPCA_AMGLUSBS(data_path, labels_path,1,300,70,l)
 
        # gamma不能大
        data_mat_new = data_mat[:,sorted_index]
        num = 20
        acc_svm = 0
        acc_knn = 0
        kappa_svm = 0
        kappa_knn = 0
        for o in range(num):
            a, b = eval_band_SVM(data_mat_new, labels_mat)
            c, d = eval_band_KNN(data_mat_new, labels_mat)
            acc_svm += a
            kappa_svm += b
            acc_knn += c
            kappa_knn += d
        acc_svm /= num
        acc_knn /= num
        kappa_svm /= num
        kappa_knn /= num
        acc_s += acc_svm
        acc_k += acc_knn
        kappa_s += kappa_svm
        kappa_k += kappa_knn
        #result_svm[i,j,k] = acc_svm
        #result_knn[i,j,k] = acc_knn
        #print('acc by svm:', acc_svm)
        #print('acc by knn:', acc_knn)
        #print('kappa by svm:', kappa_svm)
        #print('kappa by knn:', kappa_knn)
    print('acc by svm:',acc_s/10)    
    print('acc by knn:',acc_k/10)
    print('kappa by svm:',kappa_s/10)
    print('kappa by knn:',kappa_k/10)

for l in [10,15,20,25,30,35,40,45,50]:
    fun(l)

41.45672670522277
41.358312587039904
41.389853895469344
41.393007582885026
41.39901039038564
42.01424558450619
42.01079564438929
41.95965828421927
41.90079151137393
42.00439339901783
42.54998616719459
42.51603757538547
42.513566855912984
42.524981264945545
42.526684657696855
41.02562300867036
41.02178164504111
41.02174863273188
41.021503047295795
41.025822694376885
41.06375513598873
40.986986250912004
40.98978556223403
41.008186543404776
41.00728697528723
41.09358324074442
41.07398274974949
41.07056670020474
41.06368581371775
41.06389056986201
42.544128482819836
42.37817689601163
42.363576987242276
42.368371467926124
42.37160006735299
41.39702768234905
41.43796495379056
41.48355098080689
41.129476361085835
41.466798868404204
40.787755884783586
40.62185480221512
40.5538399958371
40.56423705707634
40.80292075172709
42.14790843518664
42.037048519269
42.050907211758286
42.061903317174306
42.06431596751179
acc by svm: 73.869972899729
acc by knn: 69.70644986449864
kappa by svm: 70.1379221751

In [5]:
# ONR
from work.preprocessing import preprocess_minmaxscale
from work.evaluate import evaluate_matlab
from work.bands.Indian_pines.ONR import Bands_ONR

# ONR数据采取minmax预处理
data, labels = preprocess_minmaxscale(data_path, labels_path)
a = Bands_ONR[50]
print(len(a))
evaluate_matlab(a, data, labels)


50
acc by svm: 79.49322493224932
acc by knn: 70.47804878048781
kappa by svm: 76.63568140200792
kappa by knn: 66.04183542076333


In [28]:
# TRC-OC-FDPC
# NC-OC-MVPCA
from work.evaluate import evaluate_matlab
from work.preprocessing import preprocess_normalize
from work.preprocessing import preprocess_nonzero
from work.bands.Indian_pines.TRC_OC_FDPC import Bands_TRC_OC_FDPC
from work.bands.Indian_pines.NC_OC_MVPCA import Bands_NC_OC_MVPCA

# 这两个方法数据采取正则化预处理
data, labels = preprocess_normalize(data_path, labels_path)
a = Bands_NC_OC_MVPCA[15]
print(len(a))
evaluate_matlab(a, data, labels)

15
acc by svm: 76.19728997289972
acc by knn: 71.50460704607046
kappa by svm: 72.7027890088363
kappa by knn: 67.29544954908779


In [ ]:
'''
对比方法,来自python代码
'''
from work.evaluate import evaluate_python
b = [1, 4, 9, 15, 23, 25, 26, 28, 35, 40, 52, 59, 66, 71, 73, 76, 98, 113, 130, 148, 161, 171, 177, 184]
evaluate_python(b, data_mat, labels_mat)

acc by svm: 78.65474254742547
acc by knn: 69.84281842818429


In [39]:
# MVPCA
from work.MVPCA import MVPCA
from scipy.io import loadmat
from work.evaluate import evaluate_python

# MVPCA的数据不能正则化预处理,否则影响分类精确度
data = loadmat(data_path)
index = list(data)[-1]
data = data[index]
data = data.reshape(-1,data.shape[2])
data = data.T

d = 50
band_index = MVPCA(data, d)
evaluate_python(band_index,data_mat,labels_mat)

acc by svm: 78.47967479674796
acc by knn: 70.57723577235774
kappa by svm: 75.47130313101192
kappa by knn: 66.23677927899331


In [53]:
# ISSC
from work.ISSC import ISSC_HSI
from scipy.io import loadmat
from work.estimate import eval_band_SVM
from work.estimate import eval_band_KNN
from work.preprocessing import preprocess_normalize

# 数据预处理部分只要正则化,不需要标准化
data, _ = preprocess_normalize(data_path, labels_path)
model = ISSC_HSI(40)
data_mat_new = model.predict(data)
num = 20
acc_svm = 0
acc_knn = 0
kappa_svm = 0
kappa_knn = 0
for i in range(num):
       a, b = eval_band_SVM(data_mat_new, labels_mat)
       c, d = eval_band_KNN(data_mat_new, labels_mat)
       acc_svm += a
       kappa_svm += b
       acc_knn += c
       kappa_knn += d
acc_svm /= num
acc_knn /= num
kappa_svm /= num
kappa_knn /= num
print('acc by svm:', acc_svm)
print('acc by knn:', acc_knn)
print('kappa by svm:', kappa_svm)
print('kappa by knn:', kappa_knn)


acc by svm: 79.71490514905149
acc by knn: 73.31653116531166
kappa by svm: 76.77652749537665
kappa by knn: 69.38312126690704


In [63]:
# SpaBS
from work.SpaBS import SpaBS
from work.estimate import eval_band_SVM
from work.estimate import eval_band_KNN
from work.preprocessing import preprocess_nonzero
from work.preprocessing import preprocess_normalize

data, _ = preprocess_normalize(data_path, labels_path)
model = SpaBS(50)
data_mat_new = model.predict(data)
num = 20
acc_svm = 0
acc_knn = 0
kappa_svm = 0
kappa_knn = 0
for i in range(num):
       a, b = eval_band_SVM(data_mat_new, labels_mat)
       c, d = eval_band_KNN(data_mat_new, labels_mat)
       acc_svm += a
       kappa_svm += b
       acc_knn += c
       kappa_knn += d
acc_svm /= num
acc_knn /= num
kappa_svm /= num
kappa_knn /= num
print('acc by svm:', acc_svm)
print('acc by knn:', acc_knn)
print('kappa by svm:', kappa_svm)
print('kappa by knn:', kappa_knn)


acc by svm: 68.91273712737127
acc by knn: 63.06937669376693
kappa by svm: 64.49059774613215
kappa by knn: 57.49928467247984


In [64]:
# all bands 
from work.estimate import eval_band_SVM
from work.estimate import eval_band_KNN
num = 20
acc_svm = 0
acc_knn = 0
kappa_svm = 0
kappa_knn = 0
for i in range(num):
       a, b = eval_band_SVM(data_mat_new, labels_mat)
       c, d = eval_band_KNN(data_mat_new, labels_mat)
       acc_svm += a
       kappa_svm += b
       acc_knn += c
       kappa_knn += d
acc_svm /= num
acc_knn /= num
kappa_svm /= num
kappa_knn /= num
print('acc by svm:', acc_svm)
print('acc by knn:', acc_knn)
print('kappa by svm:', kappa_svm)
print('kappa by knn:', kappa_knn)


acc by svm: 68.95934959349594
acc by knn: 63.205420054200545
kappa by svm: 64.55813337853716
kappa by knn: 57.585065350668984
